# NO2 Data Aggregation

Aggregates NO2 values by administrative boundaries and a regular hexagon grid.

# Environment Setup

In [ ]:
# Load Notebook formatter
%load_ext nb_black
# %reload_ext nb_black

In [ ]:
# Import packages
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import rasterio as rio
import geopandas as gpd
import rasterstats as rs

In [ ]:
# Set Options
# sns.set(font_scale=1.5, style="whitegrid")
# sns.set(font_scale=1.5)

In [ ]:
# Set working directory
os.chdir("..")
print(f"Working directory: {os.getcwd()}")

In [ ]:
def rasterize(vector_grid, raster_path, zonal_stats="count sum"):
    """Rasterizes values from a GeoTiff (or other
    georeferenced raster) to a polygon grid.

    Aggregates the sum by default.

    Parameters
    ----------
    vector_grid : geopandas geodataframe
        Geodataframe containing a grid of cells, sized to fit a 
        shapefile boundary.

    raster_path : str
        Path to the data the will be rasterized.

    zonal_stats : space-delimited str, optional
        Zonal statistics to calculate. Default value is 'count sum'.

    Returns
    -------
    rasterized_grid : geopandas geodataframe
        Input geodataframe with the rasterized values
        and grid centroids added.

    Example
    -------
        >>>
        >>>
        >>>
        >>>
    """
    # Open data for rasterizing
    with rio.open(raster_path) as src:

        # Extract array and metadata
        src_array = src.read(1, masked=True)
        src_meta = src.profile

    # Create copy of input geodataframe (prevents altering original)
    vector_grid_copy = vector_grid.copy()

    # Extract zonal stats (list of dictionaries)
    geojson_list = rs.zonal_stats(
        vector_grid_copy,
        src_array,
        nodata=src_meta.get("nodata"),
        affine=src_meta.get("transform"),
        geojson_out=True,
        copy_properties=True,
        stats=zonal_stats,
    )

    # Convert list to geodataframe
    rasterized_grid = gpd.GeoDataFrame.from_features(geojson_list)

    return rasterized_grid

# User-Defined Variables

In [ ]:
# Set path to path to South Korea shapefiles at levels 0, 1, and 2
#  and hexagon grid
south_korea_level_0_path = os.path.join(
    "02-raw-data", "vector", "south-korea", "gadm36_south_korea.shp"
)

south_korea_level_0 = gpd.read_file(south_korea_level_0_path)

south_korea_level_1_path = os.path.join(
    "02-raw-data", "vector", "south-korea", "gadm36_south_korea_level_1.shp"
)

south_korea_level_2_path = os.path.join(
    "02-raw-data", "vector", "south-korea", "gadm36_south_korea_level_2.shp"
)

south_korea_hexagon_grid_path = os.path.join(
    "03-processed-data",
    "vector",
    "south-korea",
    "south_korea_hexagon_grid.shp",
)

# Set path to March 2020 mean NO2 raster
no2_mean_mar_2020_path = os.path.join(
    "03-processed-data",
    "raster",
    "south-korea",
    "statistics",
    "monthly",
    "S5P-OFFL-L3-NO2-20200301-20200331-MEAN-MOL-PER-M2.tif",
)

# Data Acquisition and Preprocessing

In [ ]:
# Read shapefiles into geodataframes
south_korea_level_0 = gpd.read_file(south_korea_level_0_path)
south_korea_level_1 = gpd.read_file(south_korea_level_1_path)
south_korea_level_2 = gpd.read_file(south_korea_level_2_path)
south_korea_hexagon_grid = gpd.read_file(south_korea_hexagon_grid_path)

# Data Processing

In [ ]:
# Rasterize mean NO2 to boundaries
rasterized_level_1 = rasterize(
    vector_grid=south_korea_level_1,
    raster_path=no2_mean_mar_2020_path,
    zonal_stats="count mean",
)

rasterized_level_2 = rasterize(
    vector_grid=south_korea_level_2,
    raster_path=no2_mean_mar_2020_path,
    zonal_stats="count mean",
)

rasterized_hexagon_grid = rasterize(
    vector_grid=south_korea_hexagon_grid,
    raster_path=no2_mean_mar_2020_path,
    zonal_stats="count mean",
)

# Data Post-Processing

# Data Visualization

In [ ]:
# Plot rasterized level 1 boundaries
fig, ax = plt.subplots(figsize=(10, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
rasterized_level_1.plot(
    column="mean", ax=ax, legend=True, cax=cax, cmap="inferno"
)

plt.show()

In [ ]:
# Plot rasterized level 2 boundaries
fig, ax = plt.subplots(figsize=(10, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
rasterized_level_2.plot(
    column="mean", ax=ax, legend=True, cax=cax, cmap="inferno"
)

plt.show()

In [ ]:
# Plot rasterized hexagon boundaries
fig, ax = plt.subplots(figsize=(10, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
rasterized_hexagon_grid.plot(
    column="mean",
    ax=ax,
    legend=True,
    cax=cax,
    cmap="inferno",
    #     edgecolor="white",
    #     linewidth=3,
)
south_korea_level_0.plot(
    ax=ax, facecolor="None", edgecolor="red", linewidth=0.5
)

plt.show()

In [ ]:
# Plot rasterized hexagon boundaries
with plt.style.context("dark_background"):
    fig, ax = plt.subplots(figsize=(10, 10))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    rasterized_hexagon_grid.plot(
        column="mean",
        ax=ax,
        legend=True,
        cax=cax,
        cmap="inferno",
        edgecolor="white",
        linewidth=0.1,
    )
    south_korea_level_0.plot(
        ax=ax, facecolor="None", edgecolor="red", linewidth=0.5
    )

# Data Export